##dataset loading and shuffling


In [1]:
import pandas as pd
import numpy as np # linear algebra
import sklearn

In [2]:
df_attacking = pd.read_csv("/content/drive/MyDrive/323 final/Online dataset/attacking.csv")
df_misogyny=pd.read_csv("/content/drive/MyDrive/323 final/Online dataset/misogyny.csv")
df_offensive=pd.read_csv("/content/drive/MyDrive/323 final/Online dataset/offensive.csv")
df_racism=pd.read_csv("/content/drive/MyDrive/323 final/Online dataset/racism.csv")

In [3]:
df_all=pd.concat([df_attacking,df_misogyny,df_offensive,df_racism],ignore_index=True)

In [4]:
df_all.to_csv("concatenated.csv")

In [5]:
df_all.head()

,Text,class,class_names,class
0,`- This is not ``creative``. Those are the di...,2.0,neutral,NaN
1,` :: the term ``standard model`` is itself le...,2.0,neutral,NaN
2,"True or false, the situation as of March 200...",2.0,neutral,NaN
3,"Next, maybe you could work on being less cond...",2.0,neutral,NaN
4,This page will need disambiguation.,2.0,neutral,NaN


In [6]:
df_all=df_all[["Text","class_names"]]


In [7]:
df_all.head()

,Text,class_names
0,`- This is not ``creative``. Those are the di...,neutral
1,` :: the term ``standard model`` is itself le...,neutral
2,"True or false, the situation as of March 200...",neutral
3,"Next, maybe you could work on being less cond...",neutral
4,This page will need disambiguation.,neutral


In [8]:
df_all.reset_index(inplace=True)

In [9]:
df_all.class_names.value_counts()

neutral       124084
 neutral      102274
attacking      14782
 offensive     13590
misogyny        3377
racism          1970
Name: class_names, dtype: int64

In [10]:
df_all = df_all.dropna()
df_all = df_all.drop_duplicates()

In [11]:
df_all['class_names']=df_all.class_names.replace(' offensive','offensive')
df_all['class_names']=df_all.class_names.replace(' neutral','neutral')

In [12]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df_all.class_names)
df_all['class'] = le.transform(df_all.class_names)

In [13]:
df_all["class"].value_counts()

2    226358
0     14782
3     13590
1      3377
4      1970
Name: class, dtype: int64

In [14]:
import random

index = [i for i in range(df_all.shape[0])]
random.shuffle(index)
df_all=df_all.set_index([index]).sort_index()

In [15]:
import sklearn
df_all=sklearn.utils.shuffle(df_all)
print(df_all)

         index  ... class
128709  179724  ...     2
177279  172549  ...     2
108502  123095  ...     2
105608  160013  ...     2
27695    34686  ...     2
...        ...  ...   ...
42615   129677  ...     2
227928  205450  ...     2
102719  194996  ...     2
250888  136135  ...     2
186385  168501  ...     2

[260077 rows x 4 columns]


In [16]:
df_all.head(n=50)

,index,Text,class_names,class
128709,179724,What can I change to have the page undeleted....,neutral,2
177279,172549,"The same stalking editor, Pgagnon999 is now en...",neutral,2
108502,123095,I can't handle another round of this crap #mkr,neutral,2
105608,160013,"Wow, Klptyzm has kids?! Whats next?",neutral,2
27695,34686,` Revising estimate. I have been flipping thr...,neutral,2
166520,175668,` == Image:Arc.png listed for deletion == An ...,neutral,2
143904,13284,` == Concerts == This page should include his...,neutral,2
206513,177282,==Congratulations Gregalton. You finally go...,neutral,2
155181,66046,` Only or last warning? Make up your mind pl...,attacking,0
13278,121331,@98Halima @johnnygjokaj @cdnKhadija @rfrankh53...,neutral,2


In [17]:
df_all.reset_index(inplace=True)


In [18]:
df_all.head()

,level_0,index,Text,class_names,class
0,128709,179724,What can I change to have the page undeleted....,neutral,2
1,177279,172549,"The same stalking editor, Pgagnon999 is now en...",neutral,2
2,108502,123095,I can't handle another round of this crap #mkr,neutral,2
3,105608,160013,"Wow, Klptyzm has kids?! Whats next?",neutral,2
4,27695,34686,` Revising estimate. I have been flipping thr...,neutral,2


In [19]:
df_all["class"].value_counts()

2    226358
0     14782
3     13590
1      3377
4      1970
Name: class, dtype: int64

In [20]:
df_all = df_all.drop(df_all[df_all['class'] == 2].index)

##DATA Augmentation with spacy

In [21]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
# load nltk's English stopwords as variable called 'stop' and don't find synonym of those words.
stop = nltk.corpus.stopwords.words('english')





In [23]:
## Tokenizing sentence into token for finding synonym.
def make_tokenizer(texts):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(texts)
    return t

tokenizer = make_tokenizer(df_all['Text'])    ## Message is column name

X = tokenizer.texts_to_sequences(df_all['Text'])

from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, 70)

In [24]:


## Dictionary of word index
index_word = {}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word



In [25]:
## word list
words = [value for key, value in index_word.items()]

In [26]:
import spacy
nlp = spacy.load('en', parser=False)
def check_lemma(t,w) :
    r = [d for d in t if (nlp(d.text)[0].lemma_ != nlp(w.text)[0].lemma_)]
    return r

def get_word_synonym(word):
  filtered_words = [w for w in word.vocab if (not w.lower_ in stop) and w.is_lower == word.is_lower and w.prob >= -15] ## (not w.lower_ in stop) and
  similarity = sorted(filtered_words, key=lambda w: word.similarity(w), reverse=True)
  filtered_similarity = check_lemma(similarity[:30], word)
  return filtered_similarity[:3]

In [ ]:
## Synonym dictionary
synonym_dict = {}

for word in words:
    #if (not check_oos(word)) :
        synonym_dict.update({word : tuple([w.lower_ for w in get_word_synonym(nlp.vocab[word])])})
        #print(word, " : ", [w.lower_ for w in get_word_synonym(nlp.vocab[word])])

In [ ]:
## Only consider filtered synonym
import collections
value_occurrences = collections.Counter(synonym_dict.values())

filtered_synonym = {key: value for key, value in synonym_dict.items() if value_occurrences[value] == 1}

## Function for augmenting data by replacing words with synonym using spaCy

In [ ]:
## Function for augmenting data by replacing words with synonym using spaCy
## This might not be best best method compared to data augmentation using language translation

import re
import random
sr = random.SystemRandom()
split_pattern = re.compile(r'\s+')
def data_augmentation(message, aug_range=1) :
    augmented_messages = []
    for j in range(0,aug_range) :
        new_message = ""
        for i in filter(None, split_pattern.split(message)) :
            new_message = new_message + " " + sr.choice(filtered_synonym.get(i,[i]))
        augmented_messages.append(new_message)
    return augmented_messages

In [ ]:
## Dictionary for intent count
## Intent is column name
intent_count = df_all["class"].value_counts().to_dict()

In [ ]:
## Get max intent count to match other minority classes through data augmentation
import operator
#max_intent_count = max(intent_count.items(), key=operator.itemgetter(1))[1]
max_intent_count=218814

In [ ]:
## Loop to interate all messages
import numpy as np
import math
import tqdm
newdf = pd.DataFrame()
for intent, count in intent_count.items() :
    count_diff = max_intent_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(df_all[df_all["class"] == intent]["Text"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame([message], columns=['Text'])
            dummy1["class"] = intent
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation(message,multiplication_count)
            dummy2 = pd.DataFrame(new_messages, columns=['Text'])
            dummy2["class"] = intent
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(df_all[df_all["class"] == intent])

In [ ]:
newdf["class"].value_counts()

In [ ]:
newdf.to_csv("agumented02.csv",index=None)